In [ ]:
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.engine import Layer
from keras.layers import Activation, Add, Bidirectional, Conv1D, Dense, Dropout, Embedding, Flatten
from keras.layers import concatenate, GRU, Input, LSTM, MaxPooling1D
from keras.layers import GlobalAveragePooling1D,  GlobalMaxPooling1D, SpatialDropout1D
from keras.models import Model
import keras.backend as k
from keras.optimizers import Adam
from keras.preprocessing import text, sequence
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.test.utils import common_texts, get_tmpfile
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from smart_open import smart_open
import datetime 


import os
import time
import gc
import re
import random

In [ ]:
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.3
rate_drop_dense = 0.3

In [ ]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x


# define our own softmax function instead of K.softmax
# because K.softmax can not specify axis.
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex / K.sum(ex, axis=axis, keepdims=True)


# define the margin loss like hinge loss
def margin_loss(y_true, y_pred):
    lamb, margin = 0.5, 0.1
    return K.sum(y_true * K.square(K.relu(1 - margin - y_pred)) + lamb * (
        1 - y_true) * K.square(K.relu(y_pred - margin)), axis=-1)


class Capsule(Layer):
    """A Capsule Implement with Pure Keras
    There are two vesions of Capsule.
    One is like dense layer (for the fixed-shape input),
    and the other is like timedistributed dense (for various length input).

    The input shape of Capsule must be (batch_size,
                                        input_num_capsule,
                                        input_dim_capsule
                                       )
    and the output shape is (batch_size,
                             num_capsule,
                             dim_capsule
                            )

    Capsule Implement is from https://github.com/bojone/Capsule/
    Capsule Paper: https://arxiv.org/abs/1710.09829
    """

    def __init__(self,
                 num_capsule,
                 dim_capsule,
                 routings=3,
                 share_weights=True,
                 activation='squash',
                 **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = activations.get(activation)

    def build(self, input_shape):
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(1, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(input_num_capsule, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)

    def call(self, inputs):
        """Following the routing algorithm from Hinton's paper,
        but replace b = b + <u,v> with b = <u,v>.

        This change can improve the feature representation of Capsule.

        However, you can replace
            b = K.batch_dot(outputs, hat_inputs, [2, 3])
        with
            b += K.batch_dot(outputs, hat_inputs, [2, 3])
        to realize a standard routing.
        """

        if self.share_weights:
            hat_inputs = K.conv1d(inputs, self.kernel)
        else:
            hat_inputs = K.local_conv1d(inputs, self.kernel, [1], [1])

        batch_size = K.shape(inputs)[0]
        input_num_capsule = K.shape(inputs)[1]
        hat_inputs = K.reshape(hat_inputs,
                               (batch_size, input_num_capsule,
                                self.num_capsule, self.dim_capsule))
        hat_inputs = K.permute_dimensions(hat_inputs, (0, 2, 1, 3))

        b = K.zeros_like(hat_inputs[:, :, :, 0])
        for i in range(self.routings):
            c = softmax(b, 1)
            o = self.activation(K.batch_dot(c, hat_inputs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(o, hat_inputs, [2, 3])
                if K.backend() == 'theano':
                    o = K.sum(o, axis=1)

        return o

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)


In [ ]:
#Importing the dataset
import pandas as pd
 
dataset =pd.read_csv('Abstract_Author.csv')

#dataset.iloc[:,0:1].fillna('other', inplace=True)
#y=dataset.iloc[:,1].values

In [ ]:
k=0
X_train=[]
y_train=[]
for i in range(0,155):
  for j in range(k,k+40):
    X_train.append(dataset.iloc[j,0])
    y_train.append(dataset.iloc[j,1])
  k+=50

In [ ]:
k=40
X_test=[]
y_test=[]
for i in range(0,155):
  for j in range(k,k+10):
    X_test.append(dataset.iloc[j,0])
    y_test.append(dataset.iloc[j,1])
  k+=50

In [ ]:
X_train=pd.DataFrame(X_train)
X_test=pd.DataFrame(X_test)
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)
X_train=X_train.iloc[:,:].values
X_test=X_test.iloc[:,:].values
y_train=y_train.iloc[:,:].values
y_test=y_test.iloc[:,:].values

In [ ]:
train=np.concatenate((X_train,y_train),axis=1)
test=np.concatenate((X_test,y_test),axis=1)

In [ ]:
np.random.shuffle(train)
np.random.shuffle(test)

In [ ]:
train=pd.DataFrame(train)
test=pd.DataFrame(test)

In [ ]:
X_train=train.iloc[:,0]
X_test=test.iloc[:,0]
y_train=train.iloc[:,1]
y_test=test.iloc[:,1]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)

In [ ]:

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(encoded_Y)

In [ ]:
#!unzip "/content/drive/My Drive/1835_3176_compressed_glove.6B.100d.txt.zip"

Archive:  /content/drive/My Drive/1835_3176_compressed_glove.6B.100d.txt.zip
  inflating: glove.6B.100d.txt       


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#Tokenize the sentences
tokenizer = Tokenizer()

tokenizer.fit_on_texts(list((X_train)))

#converting text into integer sequences
X_train  = tokenizer.texts_to_sequences(X_train) 
X_test = tokenizer.texts_to_sequences(X_test)

#padding to prepare sequences of same length
X_train  = pad_sequences(X_train, maxlen=1200)
X_test = pad_sequences(X_test, maxlen=1200)

In [ ]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

20555


In [ ]:
embeddings_index = dict()
f = open('/content/glove.6B.100d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


Loaded 400000 word vectors.


In [ ]:
embedding_matrix = np.zeros((size_of_vocabulary, 100))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_eval, y_train, y_eval = train_test_split(X_train,y_train, test_size = 0.25, random_state = 0)

In [ ]:
 
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import AveragePooling1D
from keras.layers import Flatten
from keras.layers import Dense

In [ ]:
classifier = Sequential()
classifier.add(Embedding(size_of_vocabulary,100,weights=[embedding_matrix],input_length=500,trainable=False)) 
classifier.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
classifier.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
classifier.add(AveragePooling1D(3))
classifier.add(Flatten())
#classifier.add(Dense(100, activation='relu'))
classifier.add(Dense(155, activation='softmax'))
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          2055500   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 498, 128)          38528     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 496, 64)           24640     
_________________________________________________________________
capsule_1 (Capsule)          (None, 1, 1, 72)          4608      
_________________________________________________________________
flatten_1 (Flatten)          (None, 72)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 155)               11315     
Total params: 2,134,591
Trainable params: 79,091
Non-trainable params: 2,055,500
_______________________________________

In [ ]:
classifier.fit((X_train),(y_train),batch_size=128,epochs=15,validation_data=(np.array(X_eval),np.array(y_eval)),verbose=1,callbacks=[es])

Epoch 1/15
37/37 [==============================] - 20s 532ms/step - loss: 2.0823 - accuracy: 0.9273 - val_loss: 2.8422 - val_accuracy: 0.5684
Epoch 2/15
37/37 [==============================] - 20s 533ms/step - loss: 2.0046 - accuracy: 0.9357 - val_loss: 2.8241 - val_accuracy: 0.5710
Epoch 3/15
37/37 [==============================] - 20s 540ms/step - loss: 1.9345 - accuracy: 0.9381 - val_loss: 2.7624 - val_accuracy: 0.5684
Epoch 4/15
37/37 [==============================] - 20s 534ms/step - loss: 1.8646 - accuracy: 0.9469 - val_loss: 2.7223 - val_accuracy: 0.5723
Epoch 5/15
37/37 [==============================] - 20s 532ms/step - loss: 1.7815 - accuracy: 0.9544 - val_loss: 2.6781 - val_accuracy: 0.5897
Epoch 6/15
37/37 [==============================] - 20s 534ms/step - loss: 1.7097 - accuracy: 0.9572 - val_loss: 2.6413 - val_accuracy: 0.5923
Epoch 7/15
37/37 [==============================] - 20s 530ms/step - loss: 1.6434 - accuracy: 0.9606 - val_loss: 2.6041 - val_accuracy: 0.5852

In [ ]:
y_train.shape

(4773, 4)

In [ ]:
  y_pred = classifier.predict(np.array(X_test))

In [ ]:
y_pred=pd.DataFrame(y_pred)

In [ ]:
y_pred.to_csv('/content/submission_sentiment_8.csv', header=True, index=False)

In [ ]:
y_pred=pd.DataFrame(y_pred)
y_pred=y_pred.eq(y_pred.where(y_pred != 0).max(1), axis=0).astype(int)
y_pred=y_pred.iloc[:,:].values

In [ ]:
y_test=pd.DataFrame(y_test)
y_test=y_test.eq(y_test.where(y_test != 0).max(1), axis=0).astype(int)
y_test=y_test.iloc[:,:].values

In [ ]:
result=[]
for i in range(0,len(y_test)):
  for j in range(0,len(y_test[0])):
    if(y_test[i][j]==1):
      result.append(j)

In [ ]:
predicted=[]
for i in range(0,len(y_pred)):
  for j in range(0,len(y_pred[0])):
    if(y_pred[i][j]==1):
      predicted.append(j)

In [ ]:
print(result)
print(predicted)

[0, 14, 6, 28, 16, 19, 28, 16, 29, 27, 34, 41, 10, 7, 18, 27, 34, 49, 5, 29, 2, 34, 0, 6, 45, 26, 34, 48, 10, 26, 15, 11, 15, 43, 0, 31, 38, 47, 30, 12, 42, 21, 16, 17, 1, 42, 4, 19, 42, 48, 48, 39, 24, 11, 9, 24, 38, 15, 13, 16, 29, 22, 6, 36, 39, 26, 11, 37, 23, 18, 3, 13, 27, 21, 21, 40, 17, 16, 20, 35, 28, 41, 38, 13, 3, 48, 35, 32, 11, 27, 5, 28, 2, 29, 19, 8, 35, 16, 23, 42, 40, 44, 31, 30, 23, 30, 41, 37, 7, 10, 37, 29, 9, 26, 39, 16, 40, 33, 26, 45, 22, 29, 21, 30, 9, 21, 44, 14, 2, 47, 22, 22, 32, 48, 16, 38, 1, 0, 39, 49, 33, 7, 45, 10, 42, 44, 44, 20, 17, 35, 26, 47, 19, 43, 36, 38, 47, 25, 22, 26, 11, 39, 22, 46, 33, 16, 10, 8, 3, 1, 31, 13, 3, 39, 7, 1, 15, 40, 10, 13, 13, 27, 5, 14, 25, 35, 36, 32, 10, 6, 46, 1, 49, 40, 1, 43, 48, 32, 0, 29, 38, 2, 9, 5, 29, 34, 39, 7, 20, 17, 20, 45, 42, 1, 9, 43, 36, 40, 41, 7, 14, 13, 12, 24, 48, 18, 41, 37, 31, 4, 17, 35, 28, 3, 17, 22, 23, 7, 23, 30, 2, 3, 20, 4, 21, 14, 26, 10, 19, 41, 4, 7, 0, 6, 36, 18, 37, 9, 0, 45, 17, 17, 3, 45

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(result,predicted)

cm

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0, 10,  0, ...,  0,  0,  0],
       [ 0,  0,  3, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  8,  0,  0],
       [ 0,  0,  0, ...,  0, 10,  0],
       [ 0,  0,  0, ...,  0,  0, 10]])

In [ ]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
print('Confusion Matrix :')
print(cm) 
print('Accuracy Score :',accuracy_score(result, predicted)) 
print('Report : ')
print(classification_report(result, predicted)) 

Confusion Matrix :
[[ 0  0  0 ...  0  0  0]
 [ 0 10  0 ...  0  0  0]
 [ 0  0  3 ...  0  0  0]
 ...
 [ 0  0  0 ...  8  0  0]
 [ 0  0  0 ...  0 10  0]
 [ 0  0  0 ...  0  0 10]]
Accuracy Score : 0.512
Report : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.91      1.00      0.95        10
           2       0.18      0.30      0.22        10
           3       0.29      0.20      0.24        10
           4       1.00      1.00      1.00        10
           5       0.05      0.10      0.07        10
           6       0.90      0.90      0.90        10
           7       0.50      0.30      0.37        10
           8       0.17      0.10      0.12        10
           9       0.20      0.20      0.20        10
          10       0.16      0.30      0.21        10
          11       0.09      0.20      0.13        10
          12       0.40      0.20      0.27        10
          13       0.88      0.70  